# 02. ETL and SQL Schema Setup

This notebook creates the SQLite database schema and loads the cleaned data.

In [2]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import os

## 1. Load Cleaned Data

In [3]:
# First, let's run the cleaning process
df = pd.read_csv('../data/raw_data.csv', encoding='latin-1')

# Data cleaning steps
df_clean = df.copy()

# Parse dates
date_columns = ['Order Date', 'Ship Date']
for col in date_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Create date components
df_clean['order_year'] = df_clean['Order Date'].dt.year
df_clean['order_month'] = df_clean['Order Date'].dt.month
df_clean['order_quarter'] = df_clean['Order Date'].dt.quarter
df_clean['order_week'] = df_clean['Order Date'].dt.isocalendar().week
df_clean['delivery_days'] = (df_clean['Ship Date'] - df_clean['Order Date']).dt.days

# Add derived columns
df_clean['revenue'] = df_clean['Sales']
df_clean['profit_margin'] = (df_clean['Profit'] / df_clean['Sales'].replace(0, 1)) * 100

print(f"Data loaded: {len(df_clean):,} rows")

Data loaded: 51,290 rows


/var/folders/4n/xtvcccld0_5fh420f1r1rtvm0000gn/T/ipykernel_80090/1177985828.py:11: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
/var/folders/4n/xtvcccld0_5fh420f1r1rtvm0000gn/T/ipykernel_80090/1177985828.py:11: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')


In [4]:
# Create normalized tables
orders_df = df_clean[['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 
                      'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region',
                      'order_year', 'order_month', 'order_quarter', 'order_week', 
                      'delivery_days']].drop_duplicates(subset=['Order ID'])

order_items_df = df_clean[['Order ID', 'Product ID', 'Sales', 'Quantity', 'Discount', 
                           'Profit', 'Shipping Cost', 'profit_margin']]

customers_df = df_clean[['Customer ID', 'Customer Name', 'Segment']].drop_duplicates(subset=['Customer ID'])

products_df = df_clean[['Product ID', 'Product Name', 'Category', 'Sub-Category']].drop_duplicates(subset=['Product ID'])

categories_df = df_clean[['Category', 'Sub-Category']].drop_duplicates()

regions_df = df_clean[['Region', 'Country', 'State', 'City']].drop_duplicates()

# Save cleaned data
df_clean.to_csv('../data/superstore_clean.csv', index=False)
orders_df.to_csv('../data/orders_clean.csv', index=False)
order_items_df.to_csv('../data/order_items_clean.csv', index=False)
customers_df.to_csv('../data/customers_clean.csv', index=False)
products_df.to_csv('../data/products_clean.csv', index=False)

print("Tables created:")
print(f"  Orders: {len(orders_df):,} rows")
print(f"  Order Items: {len(order_items_df):,} rows")
print(f"  Customers: {len(customers_df):,} rows")
print(f"  Products: {len(products_df):,} rows")
print(f"  Categories: {len(categories_df):,} rows")
print(f"  Regions: {len(regions_df):,} rows")

Tables created:
  Orders: 25,035 rows
  Order Items: 51,290 rows
  Customers: 1,590 rows
  Products: 10,292 rows
  Categories: 17 rows
  Regions: 3,819 rows


## 2. Create SQLite Database

In [5]:
# Create database connection
db_path = '../db/superstore.db'
os.makedirs('../db', exist_ok=True)

# Remove existing database if it exists
if os.path.exists(db_path):
    os.remove(db_path)

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"Database created at: {db_path}")

Database created at: ../db/superstore.db


## 3. Create Schema

In [6]:
# SQL Schema
schema_sql = """
-- Customers table
CREATE TABLE IF NOT EXISTS customers (
    customer_id TEXT PRIMARY KEY,
    customer_name TEXT NOT NULL,
    segment TEXT
);

-- Products table
CREATE TABLE IF NOT EXISTS products (
    product_id TEXT PRIMARY KEY,
    product_name TEXT NOT NULL,
    category TEXT,
    sub_category TEXT
);

-- Orders table
CREATE TABLE IF NOT EXISTS orders (
    order_id TEXT PRIMARY KEY,
    order_date DATE,
    ship_date DATE,
    ship_mode TEXT,
    customer_id TEXT,
    segment TEXT,
    country TEXT,
    city TEXT,
    state TEXT,
    postal_code TEXT,
    region TEXT,
    order_year INTEGER,
    order_month INTEGER,
    order_quarter INTEGER,
    order_week INTEGER,
    delivery_days INTEGER,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

-- Order Items table
CREATE TABLE IF NOT EXISTS order_items (
    order_id TEXT,
    product_id TEXT,
    sales REAL,
    quantity INTEGER,
    discount REAL,
    profit REAL,
    shipping_cost REAL,
    profit_margin REAL,
    PRIMARY KEY (order_id, product_id),
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);

-- Categories table
CREATE TABLE IF NOT EXISTS categories (
    category TEXT,
    sub_category TEXT,
    PRIMARY KEY (category, sub_category)
);

-- Regions table
CREATE TABLE IF NOT EXISTS regions (
    region TEXT,
    country TEXT,
    state TEXT,
    city TEXT
);
"""

# Execute schema creation
for statement in schema_sql.split(';'):
    if statement.strip():
        cursor.execute(statement)

conn.commit()
print("Schema created successfully")

Schema created successfully


In [7]:
# Save schema to file
with open('../sql/schema.sql', 'w') as f:
    f.write(schema_sql)

print("Schema saved to sql/schema.sql")

Schema saved to sql/schema.sql


## 4. Load Data into Database

In [8]:
# Use SQLAlchemy for easier data loading
engine = create_engine(f'sqlite:///{db_path}')

# Rename columns to match database schema
customers_df.columns = ['customer_id', 'customer_name', 'segment']
products_df.columns = ['product_id', 'product_name', 'category', 'sub_category']
orders_df.columns = ['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id',
                     'segment', 'country', 'city', 'state', 'postal_code', 'region',
                     'order_year', 'order_month', 'order_quarter', 'order_week', 'delivery_days']
order_items_df.columns = ['order_id', 'product_id', 'sales', 'quantity', 'discount',
                          'profit', 'shipping_cost', 'profit_margin']
categories_df.columns = ['category', 'sub_category']
regions_df.columns = ['region', 'country', 'state', 'city']

# Load data into tables
customers_df.to_sql('customers', engine, if_exists='replace', index=False)
products_df.to_sql('products', engine, if_exists='replace', index=False)
orders_df.to_sql('orders', engine, if_exists='replace', index=False)
order_items_df.to_sql('order_items', engine, if_exists='replace', index=False)
categories_df.to_sql('categories', engine, if_exists='replace', index=False)
regions_df.to_sql('regions', engine, if_exists='replace', index=False)

print("Data loaded into database successfully")

Data loaded into database successfully


## 5. Create Indexes for Performance

In [9]:
# Create indexes
index_statements = [
    "CREATE INDEX idx_orders_date ON orders(order_date);",
    "CREATE INDEX idx_orders_customer ON orders(customer_id);",
    "CREATE INDEX idx_orders_region ON orders(region);",
    "CREATE INDEX idx_order_items_order ON order_items(order_id);",
    "CREATE INDEX idx_order_items_product ON order_items(product_id);",
    "CREATE INDEX idx_products_category ON products(category);",
    "CREATE INDEX idx_orders_year_month ON orders(order_year, order_month);"
]

for statement in index_statements:
    try:
        cursor.execute(statement)
    except sqlite3.OperationalError:
        pass  # Index already exists

conn.commit()
print("Indexes created successfully")

Indexes created successfully


## 6. Verify Database and Run Sample Queries

In [10]:
# Verify table counts
tables = ['customers', 'products', 'orders', 'order_items', 'categories', 'regions']

print("Table row counts:")
for table in tables:
    count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", conn).iloc[0]['count']
    print(f"  {table}: {count:,} rows")

Table row counts:
  customers: 1,590 rows
  products: 10,292 rows
  orders: 25,035 rows
  order_items: 51,290 rows
  categories: 17 rows
  regions: 3,819 rows


In [11]:
# Sample query 1: Top 5 customers by revenue
query1 = """
SELECT 
    c.customer_name,
    COUNT(DISTINCT o.order_id) as total_orders,
    SUM(oi.sales) as total_revenue,
    SUM(oi.profit) as total_profit
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_id, c.customer_name
ORDER BY total_revenue DESC
LIMIT 5;
"""

result1 = pd.read_sql(query1, conn)
print("Top 5 Customers by Revenue:")
print(result1)

Top 5 Customers by Revenue:
  customer_name  total_orders  total_revenue  total_profit
0  Tom Ashbrook            24    36663.11880    6328.04710
1  Tamara Chand            27    36175.36400    9392.64990
2     Greg Tran            30    36099.40528    5408.73218
3   Sean Miller            21    31148.84496   -1079.20434
4  Fred Hopkins            28    30429.21502    4478.56162


In [12]:
# Sample query 2: Monthly revenue trend
query2 = """
SELECT 
    order_year,
    order_month,
    COUNT(DISTINCT o.order_id) as orders,
    SUM(oi.sales) as revenue,
    SUM(oi.profit) as profit,
    AVG(oi.profit_margin) as avg_margin
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE order_year >= 2015
GROUP BY order_year, order_month
ORDER BY order_year DESC, order_month DESC
LIMIT 12;
"""

result2 = pd.read_sql(query2, conn)
print("\nMonthly Revenue Trend (Last 12 months):")
print(result2)


Monthly Revenue Trend (Last 12 months):
Empty DataFrame
Columns: [order_year, order_month, orders, revenue, profit, avg_margin]
Index: []


In [13]:
# Sample query 3: Category performance
query3 = """
SELECT 
    p.category,
    COUNT(DISTINCT oi.order_id) as orders,
    SUM(oi.quantity) as units_sold,
    SUM(oi.sales) as revenue,
    SUM(oi.profit) as profit,
    AVG(oi.profit_margin) as avg_margin
FROM products p
JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.category
ORDER BY revenue DESC;
"""

result3 = pd.read_sql(query3, conn)
print("\nCategory Performance:")
print(result3)


Category Performance:
          category  orders  units_sold       revenue        profit  avg_margin
0       Technology    8354       35176  4.744557e+06  663778.73318    4.967903
1        Furniture    8195       34954  4.110874e+06  285204.72380    0.859333
2  Office Supplies   19003      108182  3.787070e+06  518473.83430    5.896465


In [14]:
# Close connection
conn.close()
print("\nDatabase setup complete!")


Database setup complete!
